본문은 권창현 교수님의 줄리아 책, julia programming for Operations Rsearch를 참고하여 정리하였습니다.
> https://www.softcover.io/read/7b8eb7d0/juliabook2/frontmatter

# Chapter 10. Lagrangian Relaxation
예전에 R로 짰었는데, 이번에는 줄리아이다!
LR은 MILP 문제를 푸는데 유용하다. 키 아이디어는 hard constraint를 relax해서 좀 더 쉬운 문제를 푸는 것이다. 이 쉬운 문제들을 여러번 풀면서 Lagrangian multipliers의 적당한 값을 업데이트 한다.

## 10.1 Introduction

Fisher(2004)의 논문을 바탕으로 코드를 구현하였다.
>Fisher, Marshall L. "The Lagrangian relaxation method for solving integer programming problems." Management science 50.12_supplement (2004): 1861-1871.

다음 문제를 고려해보자.

$$Z^* = \min \mathbf{c^Tx}$$

$$
\mathbf{s.t\quad Ax=b \\
Dx \le e \\
x \ge 0, Integer.}
$$
    
$\lambda$ 는 첫번째 제약조건에 대한 dual variable이라고 하자. 첫번째 제약조건을 완화함으로써 Lagrangian problem은 다음과 같이 만들수 있다.

$$Z_D(\lambda) = \min \mathbf{c^Tx}+\lambda^T(\bf{Ax-b})$$

$$
\mathbf{s.t\quad Dx \le e \\
x \ge 0, Integer.}
$$

라그랑지안 문제는 원래 문제에 비해 쉬워진다. 간단한 LR 방법은 다음과 같이 요약할 수 있다.

- **Step 0:** 초기 람다 값인 $\lambda^0$를 예측한다.
- **Step 1:** $\lambda^k$가 주어졌을 때, 라그랑지안 문제를 풀고 $Z_D(\lambda^k)$를 구한다. 라그랑지안 문제에서 얻은 $\bf{x}^k_D$의 솔루션을 구하는데, 이는 원래 문제에서는 infeasible할 수 있다.
- **Step 2:** $\lambda^k$와 $\bf{x}^k_D$ 값이 주어졌을 때, feasible 솔루션 $\bf{x}^k$를 *어떤 벙법*으로 구한다.
- **Step 3:** *어떤 벙법*을 반복하면서 $\lambda^k$와 $\lambda^{k+1}$을 업데이트한다.

### 10.1.1 Lower and Upper Bound

원 문제를 풂으로써 우리의 목적은 목적값인 $Z^*$의 최적 값을 계산하는 것이다.우리의 목표는 알 수 없는 최적 목적값의 바운드를 위아래로 줄여나가는 것이다. 즉, 다음과 같은 관계에서
$$Z_{LB} \ge Z^* \ge Z_{UB}$$
최적 목적 값은 $Z_{LB} = Z_{UB} = Z^*$이 될 것이다. 우리는 이 사이의 갭을 optimal gap이라고 할것이며, 다음과 같이 나타낼 수 있다.

$$\mbox{Optimal Gap} = \frac{Z_{UB}-Z_{LB}}{Z_{UB}} \times 100 % $$


따라서 라그랑지안 문제는 다음과 같은 특징을 가지고 있다.

$Z_D(\lambda)\geZ^* \mbox{for all } \lambda \le 0$ , 라그랑지안 문제에서 최적 목적값은 원래 값보다 무조건 작다. (최소 문제인 경우!, 즉, 완화 해주었으니깐 더 좋은 값을 가질 수 밖에 없다는 것.)

그러므로 LR은 더 낮은 lower-bound를 제공할 수 밖에 없다. 우리는 람다를 바꿔가면서 이 lower-bound가 최대한 원래 문제와 같은 목적함수값을 구하고 싶은 것이다.


_**그렇다면 어떻게 람다를 업데이트 할 것인가?**_

뒤에서 설명할 것이다.


일단 우리는 완전성 조건 (integrality condition)을 제거한 LP relaxation이라고 불리는 relaxation 방법을 고려해 볼 수 있다. 

$$Z_{LP} = \min \mathbf{c^Tx}$$

$$
\mathbf{s.t\quad Ax=b \\
Dx \le e \\
x \ge 0}
$$


당연히 $Z_{LP}$는 원래 문제의 최적 목적값보다는 낮다. 이는 lower-bound가 될 것이다.
$$Z_{LP}\le Z^*$$

upper-bound는 임의의 feasiable solution $\bf x$로 이루어진 $\bf{c^Tx}$이 될 것이다.
$$Z^*\le\bf{c^Tx}$$

현실적인 질문은 어떻게 *좋은* upper-bound를 찾느냐 이다. 가능한 방법 중 하나로 라그랑지안 문제의 해를 이용하는 것이다. $\lambda^k$가 주어졌을때의 라그랑지안 문제를 풀면 ${\bf x}^k_D$를 얻는데, 이는 infeasible이다. 혹은 optimal솔루션을 찾을 것이다. ${\bf x}^k_D$ 값의 일부 변수를 고정하고, 다른 변수를 feasible하게 수정하거나, 휴리스틱 알고리즘을 이용해서 feasible soluion ${\bf x}^k_D$를 얻을 수 있다.


### 10.1.2 Subgradient Optimization

$Z_D(\lambda)\geZ^* \mbox{for all } \lambda \le 0$ 이므로, 다음을 구하는 것이 원래 문제의 최적 해를 구하는것이다.

$$\max_{\lambda \le 0} Z_D(\lambda)$$

이 문제를 풀기위해, subgradient optimization 방법을 사용하는데, 다음과 같이 업데이트 된다.

$$\lambda^{k+1}=\max \begin{Bmatrix} \lambda^k+t_k(\bf{Ax}^k_D-b),0 \end{Bmatrix}$$

where $t_k>0$ 이며, 이를 step size라고 한다. step size는 다음과 같이 계산된다.
$$t_k = \frac{ \theta_k ( Z_\text{UB} - Z_D({\lambda}^k) ) }
         {  || {A}{x}_D^k - b ||^2 }$$
         

## 10.2 The -$p$-Median Problem

facility location 문제를 LR로 풀어볼 것이다.
>Daskin, Mark S., and Kayse Lee Maass. "The p-median problem." Location science. Springer, Cham, 2015. 21-45.

$$
\begin{align*}
  Z^* = \min \quad & \sum_{i\in\mathcal I} \sum_{j\in\mathcal J} d_j c_{ij} x_{ij} \\
\text{s.t.} \quad
& \sum_{i\in\mathcal I} x_{ij} = 1  \qquad \forall j\in\mathcal J \\
& \sum_{i\in\mathcal I} y_i = p  \\
& x_{ij} \leq y_i \qquad \forall i\in\mathcal I, j\in\mathcal J \\
& y_i \in \{0,1\} \qquad \forall i\in\mathcal I \\
& x_{ij} \geq 0 \qquad \forall i\in\mathcal I, j\in\mathcal J
\end{align*}
$$



In [7]:
using DelimitedFiles
d, header = readdlm("./dataset&result/demand.csv", ',', header=true)
data = readdlm("./dataset&result/cost.csv", ',')
cc = data[2:end, 2:end]
c = convert(Array{Float64,2}, cc)

7×10 Array{Float64,2}:
 10.0   7.0  11.0  12.0  32.0  15.0  20.0  26.0   4.0  41.0
 13.0  17.0  31.0  37.0  21.0   5.0  13.0  15.0  14.0  12.0
  4.0  13.0  14.0  22.0   8.0  31.0  26.0  11.0  12.0  23.0
 21.0  21.0  13.0  18.0   9.0  27.0  11.0  16.0  26.0  32.0
 32.0  18.0  11.0  14.0  11.0  11.0  16.0  32.0  34.0   8.0
 15.0   9.0  13.0  12.0  14.0  15.0  32.0   8.0  12.0   9.0
 28.0  32.0  15.0   2.0  17.0  12.0   9.0   6.0  11.0   6.0

In [14]:
println(d)
println(header)

[10.0; 6.0; 20.0; 32.0; 15.0; 28.0; 3.0; 19.0; 8.0; 13.0]
AbstractString["demand"]


In [13]:
data

8×11 Array{Any,2}:
 ""      "C1"    "C2"    "C3"    "C4"  …    "C7"    "C8"    "C9"    "C10"
 "L1"  10       7      11      12         20      26       4      41     
 "L2"  13      17      31      37         13      15      14      12     
 "L3"   4      13      14      22         26      11      12      23     
 "L4"  21      21      13      18         11      16      26      32     
 "L5"  32      18      11      14      …  16      32      34       8     
 "L6"  15       9      13      12         32       8      12       9     
 "L7"  28      32      15       2          9       6      11       6     

In [15]:
typeof(cc)

Array{Any,2}